In [1]:
import numpy as np
import pandas as pd
import matplotlib 
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, f1_score, roc_curve, auc, plot_confusion_matrix, RocCurveDisplay
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from sklearn import tree

from preprocessing import *

plt.rcParams["figure.dpi"] = 60

# Carga del dataset

In [5]:
df = pd.read_csv("df_train.csv")
X = df.drop(columns=['llovieron_hamburguesas_al_dia_siguiente'])
y = df['llovieron_hamburguesas_al_dia_siguiente']
X_train, X_eval, y_train, y_eval = train_test_split(X, y, random_state=117, test_size=0.1, stratify=y.astype(str))

# Selección de modelos

Buscamos la mejor combinación de los hiperparámetros max_depth, min_samples_leaf y criterion, con 4 datasets distintos, utilizando Randomized Search.

In [6]:
X_train1 = preprocessing_arboles_1(X_train)
params = {
    'max_depth': np.arange(1, 15),
    'min_samples_leaf': np.arange(1, 300),
    "criterion": ["gini", "entropy"],
}

clf1 = tree.DecisionTreeClassifier(random_state=117)

rgscv1 = RandomizedSearchCV(
    clf1, params, n_iter = 180, scoring='roc_auc', n_jobs=-1, cv=5, return_train_score=True
).fit(X_train1, y_train)

print(f"Best score: {rgscv1.best_score_}")
print(f"Best params {rgscv1.best_params_}")

Best score: 0.8567452572514875
Best params {'min_samples_leaf': 122, 'max_depth': 9, 'criterion': 'entropy'}


In [7]:
X_train2 = preprocessing_arboles_2(X_train)
params = {
    'max_depth': np.arange(1, 15),
    'min_samples_leaf': np.arange(1, 300),
    "criterion": ["gini", "entropy"],
}

clf2 = tree.DecisionTreeClassifier(random_state=117)

rgscv2 = RandomizedSearchCV(
    clf2, params, n_iter = 180, scoring='roc_auc', n_jobs=-1, cv=5, return_train_score=True
).fit(X_train2, y_train)

print(f"Best score: {rgscv2.best_score_}")
print(f"Best params {rgscv2.best_params_}")

Best score: 0.856347769040689
Best params {'min_samples_leaf': 154, 'max_depth': 9, 'criterion': 'entropy'}


In [8]:
X_train3 = preprocessing_arboles_3(X_train)
params = {
    'max_depth': np.arange(1, 15),
    'min_samples_leaf': np.arange(1, 300),
    "criterion": ["gini", "entropy"],
}

clf3 = tree.DecisionTreeClassifier(random_state=117)

rgscv3 = RandomizedSearchCV(
    clf3, params, n_iter = 180, scoring='roc_auc', n_jobs=-1, cv=5, return_train_score=True
).fit(X_train3, y_train)

print(f"Best score: {rgscv3.best_score_}")
print(f"Best params {rgscv3.best_params_}")

Best score: 0.8500166727120112
Best params {'min_samples_leaf': 116, 'max_depth': 10, 'criterion': 'entropy'}


In [9]:
X_train4 = preprocessing_arboles_4(X_train)
params = {
    'max_depth': np.arange(1, 15),
    'min_samples_leaf': np.arange(1, 300),
    "criterion": ["gini", "entropy"],
}

clf4 = tree.DecisionTreeClassifier(random_state=117)

rgscv4 = RandomizedSearchCV(
    clf4, params, n_iter = 180, scoring='roc_auc', n_jobs=-1, cv=5, return_train_score=True
).fit(X_train4, y_train)

print(f"Best score: {rgscv4.best_score_}")
print(f"Best params {rgscv4.best_params_}")

Best score: 0.8493119094136488
Best params {'min_samples_leaf': 176, 'max_depth': 11, 'criterion': 'gini'}


# Evaluación del modelo final

El mejor modelo encontrado corresponde al preprocesamiento preprocessing_arboles_1 , con 'min_samples_leaf': 111, 'max_depth': 13, 'criterion': 'entropy'.
Entrenamos el modelo y evaluamos utilizando el set de holdout.

In [ ]:
X_eval = preprocessing_arboles_1(X_eval)

clf = tree.DecisionTreeClassifier(random_state=117, max_depth=13, min_samples_leaf=111, criterion='entropy')
clf.fit(X_train1, y_train)

y_pred = clf.predict(X_eval)
y_pred_proba = clf.predict_proba(X_eval)[:,1]

In [ ]:
print(f"Auc Roc: {roc_auc_score(y_eval, y_pred_proba)}")
print(f"Accuracy: {accuracy_score(y_eval, y_pred)}")
print(f"Precision: {precision_score(y_eval, y_pred)}")
print(f"Recall: {recall_score(y_eval, y_pred)}")
print(f"F1 Score: {f1_score(y_eval, y_pred)}")

## Matriz de confusión

In [ ]:
fig, ax = plt.subplots(figsize=(15, 7))
plt.grid(False)
plot_confusion_matrix(
    clf, X_eval, y_eval, cmap=plt.cm.Blues, display_labels=['1', '0'], ax=ax
)
plt.show()

## Curva ROC

In [ ]:
fpr, tpr, thresholds = roc_curve(y_eval, y_pred_proba)
roc_auc = auc(fpr, tpr)
display = RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc)
display.plot()
plt.show()

## Feature Importance:

In [ ]:
sorted(list(zip(X_train.columns, clf.feature_importances_)), key=lambda x: -x[1])

In [ ]:
X_train1 = preprocessing_arboles_1(X_train)

clf = tree.DecisionTreeClassifier(random_state=117, max_depth=13, min_samples_leaf=111, criterion='entropy')
clf.fit(X_train1, y_train)

df_pred = pd.read_csv("pred.csv")
df_pred_preprocess = basic_preprocessing(df_pred, prediction_dataset=True)
df_pred_preprocess = fill_numerical_missings(df_pred_preprocess, X_train)
df_pred_preprocess = df_pred_preprocess.drop(columns = ['id'])
df_pred_preprocess = preprocessing_arboles_1(df_pred_preprocess)

y_pred_final = clf.predict(df_pred_preprocess)
df_pred_final = pd.DataFrame({'id': df_pred['id'], 'llovieron_hamburguesas_al_dia_siguiente': y_pred_final})
df_pred_final.loc[df_pred_final['llovieron_hamburguesas_al_dia_siguiente'] == 1, 'llovieron_hamburguesas_al_dia_siguiente'] = 'si'
df_pred_final.loc[df_pred_final['llovieron_hamburguesas_al_dia_siguiente'] == 0, 'llovieron_hamburguesas_al_dia_siguiente'] = 'no'

df_pred_final['llovieron_hamburguesas_al_dia_siguiente'].value_counts()

In [ ]:
df_pred_final.to_csv('predicciones/arbol_de_decision.csv', index=False)